In [ ]:
!python --version

In [ ]:
!pip show google-cloud-aiplatform

# PROBAR CÓDIGOS REMOTE TRAINING
Entrenar modelos de sklearn con el iris dataset. Entrenamiento remoto de modelos de sklearn

- **Carpeta de github oficial de GCP con ejemplos de remote training**: https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/vertex_ai_sdk

- **Notebook de github de GCP replicado (entrenar modelo de sklearn)**: https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/vertex_ai_sdk/remote_training_sklearn.ipynb

In [ ]:
import vertexai
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID_DS = os.environ.get("PROJECT_GCP", "")

### 0. Definir params

In [ ]:
# PARAMS GENERALES GCP
PROJECT_ID = PROJECT_ID_DS
REGION = "{region}"
BUCKET_URI = 'gs://{bucket-id}/remote-training' # bucket creado previamente

In [ ]:
#! gcloud auth application-default login

In [ ]:
# ! gcloud config set project {PROJECT_ID}

### 1. Autenticarse
Vertex Workbrench: No es necesario
Colab / Jupyter notebooks: Es necesario autenticarse

In [ ]:
! gcloud auth login

### 2. Inicializar vertex

In [ ]:
# params
REMOTE_JOB_NAME = "remote-scalar"
REMOTE_JOB_BUCKET = f"{BUCKET_URI}/{REMOTE_JOB_NAME}"

print('REMOTE_JOB_BUCKET: ', REMOTE_JOB_BUCKET)

In [ ]:
vertexai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=REMOTE_JOB_BUCKET,
)

### 3. Preparar dataset
Se utiliza el iris dataset

In [ ]:
dataset = load_iris()

X, X_retrain, y, y_retrain = train_test_split(
    dataset.data, dataset.target, test_size=0.60, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# generar 3 datasets: train, retrain, test
print("Data size: ", len(dataset.target))
print("X_train size: ", len(X_train))
print("X_retrain size: ", len(X_retrain))
print("X_test size: ", len(X_test))

### 3. Feature transformation
Utilizando remote training

In [ ]:
# Switch to remote mode for training
vertexai.preview.init(remote=True)

#### 3.1 Execute remote job for fit_transform() on training data
Next, indicate that the StandardScalar class is to be executed remotely. Then set up the data transform and call the fit_transform() method is executed remotely.

In [ ]:
# ------> Setear como remoto la clase (en este caso de sklearn) que se va a ejecutar de forma remota.
# En el ejemplo se renombra la clase remota de vertex igual que la original local (probar si se puede asignar otro nombre a la clase, me imagino que si)

StandardScaler = vertexai.preview.remote(StandardScaler)


# ----> el resto del código se ejecuta igual que siempre. El único cambio es que la clase del modelo que se quiere ejecutar remoto debe pasar por un
# método de vertex

In [ ]:
transformer = StandardScaler()
transformer # la clase ahora es de vertex

In [ ]:
# entrenamiento y tranform de forma remota - si se corre una segunda vez en el notebook ya no se ejecuta la versión remota de vertex sino que la local
X_train = transformer.fit_transform(X_train)

# INCOMPLETO: CORRIENDO EN VERTEX WORKBRENCH FUNCIONA PERO AQUI NO, NO SÉ POR QUÉ